In [3]:
import pandas as pd

In [ ]:
import os
import time
import requests

# Set these in your environment (recommended)
# $env:REDDIT_CLIENT_ID = "..."
# $env:REDDIT_CLIENT_SECRET = "..."
# $env:REDDIT_USER_AGENT = "consumer-trend-tracker/0.1 by <your-reddit-username>"
CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
USER_AGENT = os.getenv("REDDIT_USER_AGENT", "consumer-trend-tracker/0.1")

if not CLIENT_ID or not CLIENT_SECRET:
    raise ValueError("Set REDDIT_CLIENT_ID and REDDIT_CLIENT_SECRET env vars before running.")

def get_app_token():
    auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
    data = {"grant_type": "client_credentials"}
    headers = {"User-Agent": USER_AGENT}
    resp = requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=data, headers=headers, timeout=30)
    resp.raise_for_status()
    return resp.json()["access_token"]

def fetch_subreddit_posts(subreddit, limit=100):
    token = get_app_token()
    headers = {"Authorization": f"bearer {token}", "User-Agent": USER_AGENT}
    params = {"limit": limit}
    url = f"https://oauth.reddit.com/r/{subreddit}/new"
    resp = requests.get(url, headers=headers, params=params, timeout=30)
    resp.raise_for_status()
    return resp.json()["data"]["children"]

def posts_to_df(posts):
    rows = []
    for p in posts:
        d = p.get("data", {})
        rows.append({
            "id": d.get("id"),
            "created_utc": d.get("created_utc"),
            "subreddit": d.get("subreddit"),
            "title": d.get("title"),
            "selftext": d.get("selftext"),
            "score": d.get("score"),
            "num_comments": d.get("num_comments"),
            "url": d.get("url"),
        })
    return pd.DataFrame(rows)

posts = fetch_subreddit_posts("technology", limit=50)
df = posts_to_df(posts)
df.head()